In [381]:
import tkinter as tk
from tkinter import messagebox, filedialog, ttk
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

Now its time to make the file explorer nicer and give it a nice layout. First python script lets you search, get the file path and can pick up CSV and XLSX files. 
Hopefully this file will do the same - show the contents of the file and then we will look at uplaoding it to a database.

In [382]:
root = tk.Tk() #widget with a title bar - intialised
root.geometry("500x500") #root starting geometry, 500 by 500 pixels
root.pack_propagate(False) #don't repack yourself based on the things that are placed in there
root.resizable(0,0) #don't resize

''

In [383]:
frame1 = tk.LabelFrame(root, text = "Excel Data")
frame1.place(height=250, width=500) #you can pack place, or grid into the widget. Pack just shoves it in any old way, grid makes 
#put your widget as a grid and select a location, place requires the specific pixels you want to cover

In [384]:
def open():
    filename = filedialog.askopenfilename(initialdir=r"C:\Users\mmje5\Desktop\Coding Self Taught\Python File Explorer", title = "Select a File", filetypes=(("excel files", "*.xlsx"),("csv files", "*.csv")))
    #now we are goign to check if the file is an excel or a csv and open it accordingly
    label_file["text"] = filename
    return None
    



    

In [390]:
def Load_Excel_Data():
    global df
    filepath = label_file["text"]
    try:
        if filepath[-1] == 'x':
            print("This is an excel file")
            excel_filename = r"{}".format(filepath)
            df = pd.read_excel(excel_filename)
        else:
            print("This is a CSV file")
            csv_filename = r"{}".format(filepath)
            df = pd.read_csv(csv_filename)
    except ValueError:
        tk.messagebox.showerror("Information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("Information", f"No such file {filepath} exists")
        return None
        
        
    clear_data()
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    for column in tv1["columns"]:
        tv1.heading(column, text=column)
        
    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("","end", values=row)
    return df
    

def upload_data():
    try:
          cnx = mysql.connector.connect(user='root', host='127.0.0.1', database='smart_van')
        #defaul wamp server creates server on 127.0.0.1, username is root and password is empty (by default)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        cursor = cnx.cursor()
        for index, row in df.iterrows():
            c = row[0]
            i = str(row[1])
            ef = str(row[2])
            query = "INSERT INTO uploadtest (Cars, ID, EnginefaultID) VALUES ('"+ c + "'," + i +", "+ ef + ")"
            cursor.execute(query)
            # Make sure data is committed to the database
            cnx.commit()
            
            #print(row[0])
            #print(type(row[0]))
            
        cursor.close()
        cnx.close()
        print("Upload Succesful")

In [386]:
def clear_data():
    tv1.delete(*tv1.get_children())

In [387]:
add_test = ("INSERT INTO uploadtest "
               "(Cars, ID, EnginefaultID)"
               "VALUES (%s, %s, %s)")

    
    

In [388]:
#frame for open file dialog
file_frame = tk.LabelFrame(root, text="Open file")
file_frame.place(height=100, width = 500, rely=0.65,relx=0)

button1 = tk.Button(file_frame, text="Browse a File", command = open)
button1.place(rely=0.65, relx=0.7)

button2 = tk.Button(file_frame, text="Load File", command= Load_Excel_Data)
button2.place(rely=0.65, relx=0.4)

button3 = tk.Button(file_frame, text="Upload to DB", command= upload_data) #make database upload command
button3.place(rely =0.65, relx = 0.1)

label_file = ttk.Label(file_frame, text = "No File Selected") #ttk has newer widgets
label_file.place(relx=0,rely=0)

#tree view widget - how to visualise dataframe
tv1 = ttk.Treeview(frame1)
tv1.place(relheight = 1, relwidth = 1) #place inside frame_1 as the conatiner and fill the whole conatiner

#now add a scroll bar
treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview)
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview)
tv1.configure(xscrollcommand =treescrollx.set, yscrollcommand = treescrolly.set)
treescrollx.pack(side="bottom", fill="x")
treescrolly.pack(side ="right", fill="y")

In [389]:
root.mainloop() # main loop starts the GUI - won't appear till this is called

This is an excel file
